### Computational Guided Inquiry for Modeling Earth's Climate (Neshyba, 2024)

# Cambio 2.0

## Equations of motion of Cambio2.0
The equations of motion of Cambio2.0 are almost the same as Cambio1.0. The sole difference is that Cambio2.0 takes into account the *Henry effect*, by which the temperature dependence of the solubility of carbon dioxide in the ocean is taken into account.

$$
F_{land->atm} =  k_{la} \ \ \ (1) 
$$

$$
F_{atm->land} = k_{al0} +  k_{al1} [C_{atm}] \ \ \ (2)
$$

$$
F_{ocean->atm} = k_{oa} \times (1+DC\times T_{anomaly}) [C_{ocean}] \ \ \ (3)
$$

$$
F_{atm->ocean} = k_{ao} [C_{atm}] \ \ \ (4)
$$

$$
F_{human->atm} = \epsilon(t) \ \ \ (5)
$$

This innovation is a first step toward incorporating feedbacks into Cambio (of which more to come later).

## The *MECLib* library and other resources
We've packed a lot of functions that you're familiar with into a resources library, *Conn350Lib*. Here's a partial list of them:

- Diagnose_OceanSurfacepH  
- Diagnose_T_anomaly  
- Diagnose_actual_temperature  
- Diagnose_degreesF  

We're also introducing a new dictionary to contain variables that define the current state of the climate (called *ClimateState*), and new functions that solve the Cambio equations of motion (called *PropagateClimateState_Cambio1* and *PropagateClimateState_Cambio2*).

## Uploading your climate emissions scenario
As before, you'll need to upload a climate emissions scenario file (like "Peaks_in_2040.hdf5", or whichever one you prefer) to the Cambio2.0 folder.

## Learning goals
1. I can make effective use of Python's "help" command.
1. I can explain how the *Henry effect* plays a role in these equations, and in their results.
1. I can make lists of Python dictionaries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5io
import sys; sys.path.append('/home'); import MECLib as CL
from copy import copy as makeacopy

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams['font.size'] = 18

### Python's "help" command
The cell below shows how to get details about functions. Try it!

In [ ]:
help(np.log10)
help(CL.Diagnose_OceanSurfacepH)

### Your turn
Use Python's "help" command to get help on a couple of functions in the CL library (check out the list in the Introduction).

In [ ]:
# your code here 


### Loading the scheduled flow
In the cell below, do what you did before in DiagnosticFunctions -- i.e., load in the scheduled flows file you have uploaded. It'll be most convenient if you use the following naming convention: 

    time, eps, epsdictionary = CL.GetMyScenario('...')

(but of course supplying the name of your own scheduled flows file).

In [ ]:
# your code here 


### Creating a dictionary for climate parameters
In the cell below, we use the CreateClimateParams function to create a dictionary of climate parameters.

In [ ]:
ClimateParams = CL.CreateClimateParams(epsdictionary)
display(ClimateParams)

### Creating a climate state

In [ ]:
ClimateState = CL.CreateClimateState(ClimateParams)
display(ClimateState)

### Making a list of climate states
Eventually, we're going to want to make list of climate states. In the next cell, we practice that process by making a very simple and boring list -- two identical preindustrial climate states. 

You might notice that the syntax for this is different from how we appended to make lists before. That's because here we're creating a regular Python list, whereas before we were creating a list of numpy arrays.

In [ ]:
ClimateState_list = []
ClimateState_list.append(ClimateState)
ClimateState_list.append(ClimateState)
display(ClimateState_list)

### Extracting a time series from a climate list
The "display" capability we just used is handy to get a view of the contents of a list, but sometimes we're more interested in particular items in the list. The function CollectClimateTimeSeries does just that. In the cell below, you can see how it works. Of course there are just two values, but when you use CollectClimateTimeSeries later, there will be hundreds of values.

In [ ]:
time_array = CL.CollectClimateTimeSeries(ClimateState_list,'time')
print(time_array)

### Your turn
Collect and print all the atmospheric reservoir carbon amounts (C_atm) in ClimateState_list.

In [ ]:
# your code here 


### Duplicating the Cambio1.0 algorithm using Python dictionaries
The cell below duplicates Cambio1.0's algorithm for propagating the climate state forward, but implemented as a Python function. This is all done for you, but it's worth taking a few minutes to inspect the code so you understand how it works.

One item in particular to note: with this architecture, the array of climate states stored as a list is one shorter than the original. That's why the time plots have "time_array" on the x-axis (instead of "time").

In [ ]:
def PropagateClimateState_Cambio1(previousClimateState, ClimateParams, dt, F_ha):
    """Propagates the state of the climate, with a specified anthropogenic carbon flux"""
    """Returns a new climate state"""

    # Extract constants from ClimateParams
    k_la = ClimateParams['k_la']
    k_al0 = ClimateParams['k_al0']
    k_al1 = ClimateParams['k_al1']
    k_oa = ClimateParams['k_oa']
    k_ao = ClimateParams['k_ao']
    
    # Extract concentrations, albedo, etc, from the previous climate state
    C_atm = previousClimateState['C_atm']
    C_ocean = previousClimateState['C_ocean']
    albedo = previousClimateState['albedo']
    time = previousClimateState['time']
    
    # Get new fluxes (including the effect of temperature anomaly on the ocean-to-atmosphere flux)
    F_la = k_la    
    F_al = k_al0 + k_al1*C_atm
    F_oa = k_oa*C_ocean    
    F_ao = k_ao*C_atm

    # Get new concentrations of carbon that depend on the fluxes
    C_atm += (F_la + F_oa - F_ao - F_al + F_ha)*dt
    C_ocean += (F_ao - F_oa)*dt
    time += dt
    
    # Create a new climate state with these updates
    ClimateState = makeacopy(previousClimateState)
    ClimateState['C_atm'] = C_atm
    ClimateState['C_ocean'] = C_ocean
    ClimateState['F_al'] = F_al
    ClimateState['F_la'] = F_la
    ClimateState['F_ao'] = F_ao
    ClimateState['F_oa'] = F_oa
    ClimateState['F_ha'] = F_ha
    ClimateState['time'] = time

    # Return the new climate state
    return ClimateState

# Running Cambio1.0

# Make the starting state the preindustrial
ClimateState = CL.CreateClimateState(ClimateParams)

# Initialize our list of climate states 
ClimateState_list = []

# The time interval
dt = time[1]-time[0]

# Loop over all the times after the initial one in the scheduled flow
for i in range(1,len(time)):

    # Propagate
    ClimateState = PropagateClimateState_Cambio1(ClimateState,ClimateParams,dt,eps[i])
    
    # Add to our list of climate states
    ClimateState_list.append(ClimateState)

# Extracting data from ClimateState_list
time_array = CL.CollectClimateTimeSeries(ClimateState_list,'time')
C_atm_array = CL.CollectClimateTimeSeries(ClimateState_list,'C_atm')
C_ocean_array = CL.CollectClimateTimeSeries(ClimateState_list,'C_ocean')
F_ao_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_ao')
F_oa_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_oa')
F_al_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_al')
F_la_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_la')
F_ha_array = CL.CollectClimateTimeSeries(ClimateState_list,'F_ha')

# Plotting the concentrations (C_atm and C_ocean) on one graph, in GtC
plt.figure()
plt.plot(time_array,C_atm_array,'red',label='C_atm')
plt.plot(time_array,C_ocean_array,label='C_ocean')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("GtC")
plt.title('Cambio1.0 results')
plt.legend()

# Plot the atmospheric concentration (C_atm), in ppm (by dividing C_atm by 2.12)
plt.figure()
plt.plot(time_array,C_atm_array/2.12,'red',label='C_atm')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("ppm")
plt.legend()
plt.title('Cambio1.0 results')

# Plot the net fluxes
plt.figure()
F_land_net = F_la_array-F_al_array
F_ocean_net = F_oa_array-F_ao_array
plt.plot(time_array,F_ha_array,label='F_ha',color='black')
plt.plot(time_array,F_land_net,label='F_la-F_al',color='brown')
plt.plot(time_array,F_ocean_net,label='F_oa-F_ao',color='blue')
plt.grid(True)
plt.xlabel('time (years)')
plt.ylabel("Flux differences (GtC/year)")
plt.legend()
plt.title('Cambio1.0 results')

### Upgrading to Cambio2.0
Below, the idea is to tweak what we just did such that we include Henry's effect (Eq. 3 in the Introduction). It's highly recommended that you make these tweaks one at a time, with plenty of testing after each one.

Here are the changes we'll need in the cell below:
1. Don't forget to name your functio  PropagateClimateState_Cambio2 (not PropagateClimateState_Cambio1), and to invoke it (not PropagateClimateState_Cambio1) in your Euler loop.
1. Inside PropagateClimateState_Cambio2, use the CL library's CL.Diagnose_T_anomaly function to calculate the temperature anomaly, and use that to Implement Henry's effect -- as represented by Eq. 3 in the Introduction; you'll have to retrieve the climate parameter ClimateParams['DC'] to do this. Also compute the actual temperature (using CL.Diagnose_actual_temperature) and the ocean surface pH (using CL.Diagnose_OceanSurfacepH) at each time step.
1. After the Euler loop, in addition to plotting the timelines you made for Cambio1.0, also plot timelines of:
- the temperature anomaly
- the actual temperature, and 
- the ocean surface pH. 

To keep things organized, it would be best to title these graphs something like "Cambio2.0 results".

In [ ]:
# your code here 


### Pause for analysis
1. Make an assessment of the impact Henry's effect made on the atmospheric amount of carbon. To do that, you can compare the peak amounts of C_atm -- either in GtC or in ppm -- for the two algorithms. Which results in a higher atmospheric peak carbon?
1. What is the peak temperature according to your Cambio2.0 results? In what year does it occur? How much of a time lag is there between peak emissions and peak temperature?
1. What's the most acidic pH?

YOUR ANSWER HERE

### Refresh/save/validate
Double-check everything is OK, and press the "Validate" button (as usual).

### Close/submit/logout
Close, submit, and log out.